In [1]:
import boto3
import json
import botocore
from IPython.display import display, Markdown
import time

In [4]:
# Initialize the bedrock client

session = boto3.session.Session()
region =  session.region_name
bedrock = boto3.client(service_name='bedrock-runtime', region_name=region)

#defining the different available models

MODELS = {
  "Claude 3.7 Sonnet": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
  "Claude 3.5 Sonnet": "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
  "Claude 3.5 Haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
  "Amazon Nova Pro": "us.amazon.nova-pro-v1:0",
  "Amazon Nova Micro": "us.amazon.nova-micro-v1:0",
  "DeepSeek-R1": "us.deepseek.r1-v1:0",
  "Meta Llama 3.1 70B Instruct": "us.meta.llama3-1-70b-instruct-v1:0"
}

In [17]:
#utility function to display the model response
def display_response(response, model_name = None):
    if model_name:
      display(Markdown(f"### Response from **{model_name}**:\n\n"))
    display(Markdown(response))
    print("\n"+"-"*80 + "\n")

In [26]:
def multiply(a, b):
    return  { "result": a * b }

multiply_tool = {
    "tools": [
        {
            "toolSpec": {
                "name": "multiply",
                "description": "Multiplies two numbers",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "a": {
                                "type": "number",
                                "description": "The first number to multiply"
                            },
                            "b": {
                                "type": "number",
                                "description": "The second number to multiply"
                            }
                        },
                        "required": ["a", "b"]
                    }
                }
            }
        }
    ],
    "toolChoice": {
        "auto": {}
    }
}

In [27]:
prompt = f"""
What is 3 times 4?
"""

converse_request = {
  "messages": [
    {
      "role": "user",
      "content": [{"text": prompt}]
    }
  ],
  "inferenceConfig": {
    "temperature": 0.0,
    "maxTokens": 500,
  }
}

In [28]:
response = bedrock.converse(
  modelId=MODELS["Claude 3.7 Sonnet"],
  messages=converse_request["messages"],
  inferenceConfig=converse_request["inferenceConfig"],
  toolConfig=multiply_tool
)

print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "ea5e25b7-0b3e-41b5-a8d0-1e161b31c5a1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 26 Jun 2025 18:09:28 GMT",
      "content-type": "application/json",
      "content-length": "456",
      "connection": "keep-alive",
      "x-amzn-requestid": "ea5e25b7-0b3e-41b5-a8d0-1e161b31c5a1"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "I can help you calculate 3 times 4 using the multiply function."
        },
        {
          "toolUse": {
            "toolUseId": "tooluse_INWyxROgR2ansZ2DseKkHA",
            "name": "multiply",
            "input": {
              "a": 3,
              "b": 4
            }
          }
        }
      ]
    }
  },
  "stopReason": "tool_use",
  "usage": {
    "inputTokens": 415,
    "outputTokens": 86,
    "totalTokens": 501,
    "cacheReadInputTokens": 0,
    "cacheWriteInputTokens": 0
  },
  "metri

In [29]:
print(response["output"]["message"]["content"][1])

{'toolUse': {'toolUseId': 'tooluse_INWyxROgR2ansZ2DseKkHA', 'name': 'multiply', 'input': {'a': 3, 'b': 4}}}


In [30]:
def handle_function_call(model_id, converse_request, tool_config):
  response = bedrock.converse(
    modelId=MODELS["Claude 3.7 Sonnet"],
    messages=converse_request["messages"],
    inferenceConfig=converse_request["inferenceConfig"],
    toolConfig=multiply_tool
  )

  content_blocks = response["output"]["message"]["content"]
  has_tool_use = any("toolUse" in block for block in content_blocks)
  if has_tool_use:
    tool_use  = next(block for block in content_blocks if "toolUse" in block)
    tool_name = tool_use["toolUse"]["name"]
    tool_input = tool_use["toolUse"]["input"]
    tool_id = tool_use["toolUse"]["toolUseId"]

    if tool_name == "multiply":
      a = int(tool_input["a"])
      b = int(tool_input["b"])
      result = multiply(a, b)
      updated_messages = converse_request["messages"] + [
        {
          "role": "assistant",
          "content": [
            {
              "toolUse": {
                "toolUseId": tool_id,
                "name": tool_name,
                "input": tool_input
              }
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "toolResult": {
                "toolUseId": tool_id,
                "content": [
                  {
                    "json": result
                  }
                ],
                "status": "success"
              }
            }
          ]
        }
      ]
      
      final_response = bedrock.converse(
        modelId=model_id,
        messages=updated_messages,
        inferenceConfig=converse_request["inferenceConfig"],
        toolConfig=tool_config
      )

      final_text = ""
      for block in final_response["output"]["message"]["content"]:
          if "text" in block:
              final_text = block["text"]
              break
      
      return {
          "tool_call": {"name": tool_name, "input": tool_input},
          "tool_result": result,
          "final_response": final_text
      }
  else:
    final_text = ""
    for block in response["output"]["message"]["content"]:
        if "text" in block:
            final_text = block["text"]
            break
    return {
        "tool_call": None,
        "tool_result": None,
        "final_response": final_text
    }


In [31]:
function_result = handle_function_call(
    MODELS["Claude 3.7 Sonnet"], 
    converse_request,
    multiply_tool
)

# Display the results
if "error" not in function_result:
    if "tool_call" in function_result:
        print(f"Tool Call: {function_result['tool_call']['name']}({function_result['tool_call']['input']})")
        print(f"Tool Result: {function_result['tool_result']}")
    
    display_response(function_result["final_response"], "Claude 3.7 Sonnet (Function Calling)")
else:
    print(f"Error: {function_result['error']}")

Tool Call: multiply({'a': 3, 'b': 4})
Tool Result: {'result': 12}


### Response from **Claude 3.7 Sonnet (Function Calling)**:



The result of 3 times 4 is 12.


--------------------------------------------------------------------------------

